# Quantzig Data Engineer Case-Study:

1. Import Required Libraries

In [1]:
import pandas as pd
import re
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

2. Loading Data-DE,Rules-DE Excel files 

In [2]:
# Load the dataset
data_file = r"C:\Users\NARENDRA MEENA\Downloads\Data - DE.xlsx"
data_df = pd.read_excel(data_file, sheet_name='Sheet1')
# Load the rules
rules_file = r"C:\Users\NARENDRA MEENA\Downloads\Rules - DE.xlsx"
rules_df = pd.read_excel(rules_file, sheet_name='Rules')

In [3]:
data_df

,ID,Date,Quantity,Company Name,Price$,Description,Sales#
0,38588885,2022-01-01,48,CompanyName,10,BRAND1,480
1,38600047,2022-01-01,196,CompanyName,10,BRAND1,1960
2,38600068,2022-01-01,30,CompanyName,10,BRAND2,300
3,38600070,2022-01-01,48,CompanyName,10,BRAND1,480
4,38600073,2022-01-01,60,CompanyName,10,BRAND1,600
...,...,...,...,...,...,...,...
52574,38752885,2022-01-24,192,CompanyName,20,BRAND1,3840
52575,38752888,2022-01-24,144,CompanyName,20,BRAND1,2880
52576,38752803,2022-01-24,240,CompanyName,20,BRAND1,4800
52577,38752802,2022-01-24,384,CompanyName,20,BRAND1,7680


In [4]:
rules_df

,Pillar Name (EN),Sheet Name (TR),Column name (required),Data type,Nullable
0,Storage,Data,ID,String,N
1,Storage,Data,Date,Date,N
2,Storage,Data,Quantity,Integer,N
3,Storage,Data,Company,String,N
4,Storage,Data,Price,Integer,N
5,Storage,Data,Description,String,N
6,Storage,Data,Sales,Decimal - Money,N


3. Write Rule Checks for Data-DE based on Rules-DE

In [5]:
data_quality_issues = []

for column in data_df.columns:
    if re.search(r'[^a-zA-Z0-9_]', column):
        data_quality_issues.append(f"Column name '{column}' contains special characters or spaces.")

for index, row in rules_df.iterrows():
    col_name = row['Column name (required)']
    expected_dtype = row['Data type']
    nullable = row['Nullable']
    
    if col_name not in data_df.columns:
        data_quality_issues.append(f"Column '{col_name}' is missing.")
        continue
    
    if not nullable and data_df[col_name].isnull().any():
        data_quality_issues.append(f"Column '{col_name}' contains null values.")
    
    if expected_dtype == 'String':
        if not all(isinstance(val, str) for val in data_df[col_name] if pd.notnull(val)):
            data_quality_issues.append(f"Column '{col_name}' contains non-string values.")
    elif expected_dtype == 'Integer':
        if not all(isinstance(val, int) for val in data_df[col_name] if pd.notnull(val)):
            data_quality_issues.append(f"Column '{col_name}' contains non-integer values.")
    elif expected_dtype == 'Date':
        try:
            pd.to_datetime(data_df[col_name], format='%Y-%m-%d')
        except ValueError:
            data_quality_issues.append(f"Column '{col_name}' contains invalid date formats.")
    elif expected_dtype == 'Decimal - Money':
        if not all(isinstance(val, (int, float)) for val in data_df[col_name] if pd.notnull(val)):
            data_quality_issues.append(f"Column '{col_name}' contains non-decimal values.")


In [6]:
data_quality_issues

["Column name 'Company Name' contains special characters or spaces.",
 "Column name 'Price$' contains special characters or spaces.",
 "Column name 'Sales#' contains special characters or spaces.",
 "Column 'ID' contains non-string values.",
 "Column 'Company' is missing.",
 "Column 'Price' is missing.",
 "Column 'Sales' is missing."]

4. Impleamenting Issues Correction in Data-DE

In [7]:
data_df.columns = [re.sub(r'[^a-zA-Z0-9_]', '', col) for col in data_df.columns]

for index, row in rules_df.iterrows():
    col_name = row['Column name (required)']
    expected_dtype = row['Data type']
    
    if col_name in data_df.columns:
        data_df[col_name] = data_df[col_name].apply(lambda x: x.strip() if isinstance(x, str) else x)
        
        if expected_dtype == 'String':
            data_df[col_name] = data_df[col_name].astype(str)
        elif expected_dtype == 'Integer':
            data_df[col_name] = pd.to_numeric(data_df[col_name], errors='coerce').fillna(0).astype(int)
        elif expected_dtype == 'Date':
            data_df[col_name] = pd.to_datetime(data_df[col_name], format='%Y-%m-%d', errors='coerce')
        elif expected_dtype == 'Decimal - Money':
            data_df[col_name] = pd.to_numeric(data_df[col_name], errors='coerce').fillna(0.0)


In [ ]:
cleaned_file = 'Cleaned_DATA-DE.xlsx'
data_df.to_excel(cleaned_file, index=False)

After Correcting Columns based on Rules-DE 

In [8]:
data_df

,ID,Date,Quantity,CompanyName,Price,Description,Sales
0,38588885,2022-01-01,48,CompanyName,10,BRAND1,480
1,38600047,2022-01-01,196,CompanyName,10,BRAND1,1960
2,38600068,2022-01-01,30,CompanyName,10,BRAND2,300
3,38600070,2022-01-01,48,CompanyName,10,BRAND1,480
4,38600073,2022-01-01,60,CompanyName,10,BRAND1,600
...,...,...,...,...,...,...,...
52574,38752885,2022-01-24,192,CompanyName,20,BRAND1,3840
52575,38752888,2022-01-24,144,CompanyName,20,BRAND1,2880
52576,38752803,2022-01-24,240,CompanyName,20,BRAND1,4800
52577,38752802,2022-01-24,384,CompanyName,20,BRAND1,7680


5. Impleamenting Email Sending Feature for Issues for given Data-DE:

In [ ]:
def send_email(subject, body, to_email):
    from_email = "username@email.com"
    from_password = "password"

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()
    server.login(from_email, from_password)
    text = msg.as_string()
    server.sendmail(from_email, to_email, text)
    server.quit()
    
issues_message = "\n".join(data_quality_issues)
send_email(subject="Data Quality Issues", body=issues_message, to_email="receiver@email.com")